What we discussed:
1. More words in the intro, but still keep theme of Rabi oscillations.
2. Supplementary: show experimental unresolvability of 3rd branch (investigate those fringes)
3. Plot $\chi^2(\Omega_0)$ for all $4$ fits.
4. See Yunpeng's Fermionic-Joule Thomson effect paper for more ideas on how to structure the paper.

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import LogFormatterMathtext, ScalarFormatter

import scienceplots
plt.style.use(['science'])

from scipy.fft import ifft, fftfreq, rfftfreq, rfft
from scipy.optimize import curve_fit, least_squares

import re
from pathlib import Path

import pandas as pd

In [2]:
# Note, M_inf goes to 0 when detuned repulsively, whereas M_inf goes to 1 when detuned attractively

def y1(x, M_inf, A_1, Gamma_1, Omega_1, gamma):
    return M_inf - A_1 * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x) - (1 - A_1 + M_inf) * np.exp(-gamma*x/2)

def y2(x, M_inf, Gamma_1, Omega_1):
    return M_inf - (1 + M_inf) * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x)

def y3(x, M_inf, A_1, Gamma_1, Omega_1, A_2, Gamma_2, Omega_2, gamma):
    return M_inf - A_1 * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x) - A_2 * np.exp(-Gamma_2*x/2) * np.cos(Omega_2*x) - (1 - A_1 - A_2 + M_inf) * np.exp(-gamma*x/2)

def y4(x, M_inf, A_1, Gamma_1, Omega_1, Gamma_2, Omega_2):
    return M_inf - A_1 * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x) - (1-A_1+M_inf) * np.exp(-Gamma_2*x/2) * np.cos(Omega_2*x)

In [3]:
dir_path = "C:\\Users\\weidu\\OneDrive\\Documents\\Personal STEM Projects\\Rabi\\time_series_data\\invkFa_-0.01_a\\"
df = pd.read_csv(dir_path + "RabiOsc_Omega0EF_0.19.csv").to_numpy()
print(df)

[[ 0.00000000e+00  0.00000000e+00 -1.02487170e+00  2.74251990e-02]
 [ 9.80000000e+01  3.33737671e+00 -8.82730375e-01  6.04507999e-03]
 [ 1.98000000e+02  6.74286314e+00 -5.75246396e-01  2.38379895e-02]
 [ 2.98000000e+02  1.01483496e+01 -2.59377257e-01  3.98700559e-03]
 [ 3.98000000e+02  1.35538360e+01  6.58162853e-02  5.09096761e-02]
 [ 4.98000000e+02  1.69593225e+01  3.16674331e-01  2.57385893e-02]
 [ 5.98000000e+02  2.03648089e+01  3.27935640e-01  1.30887446e-02]
 [ 6.98000000e+02  2.37702953e+01  2.97615988e-01  4.73912799e-02]
 [ 7.98000000e+02  2.71757818e+01  1.29526259e-01  1.49802490e-02]
 [ 8.98000000e+02  3.05812682e+01  7.34851622e-02  1.14748186e-02]
 [ 9.98000000e+02  3.39867546e+01 -8.03938149e-02  2.51137196e-02]
 [ 1.09800000e+03  3.73922411e+01 -1.07200848e-01  4.26589292e-03]
 [ 1.19800000e+03  4.07977275e+01 -1.05535069e-01  2.88776787e-02]
 [ 1.29800000e+03  4.42032139e+01 -1.03124692e-01  2.92297721e-03]
 [ 1.39800000e+03  4.76087004e+01 -5.52942880e-02  2.55439385e

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from ipywidgets import interact, FloatSlider
import inspect

# --- 1. Define your models ---
def y1(x, M_inf, A_1, Gamma_1, Omega_1, gamma):
    # Note: Enforces y(0) = -1
    return M_inf - A_1 * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x) \
           - (1 - A_1 + M_inf) * np.exp(-gamma*x/2)

def y2(x, M_inf, Gamma_1, Omega_1):
    return M_inf - (1 + M_inf) * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x)

def y3(x, M_inf, A_1, Gamma_1, Omega_1, A_2, Gamma_2, Omega_2, gamma):
    return M_inf - A_1 * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x) \
        - A_2 * np.exp(-Gamma_2*x/2) * np.cos(Omega_2*x) \
            - (1 - A_1 - A_2 + M_inf) * np.exp(-gamma*x/2)

def y4(x, M_inf, A_1, Gamma_1, Omega_1, Gamma_2, Omega_2):
    return M_inf - A_1 * np.exp(-Gamma_1*x/2) * np.cos(Omega_1*x) - (1-A_1+M_inf) * np.exp(-Gamma_2*x/2) * np.cos(Omega_2*x)

# --- 2. Load Data ---
# Note: Ensure this path is correct on your local machine
path = "C:\\Users\\weidu\\OneDrive\\Documents\\Personal STEM Projects\\Rabi\\time_series_data\\invkFa_-0.01_a\\RabiOsc_Omega0EF_4.98.csv" 
data = pd.read_csv(path, header=None, skiprows=1).to_numpy()
t_data = data[:, 1]
M_data = data[:, 2]
M_err = data[:, 3]

# --- 3. The Interactive Tool (Fixed with Bounds) ---
def manual_fit_tool_unweighted(model_func):
    
    # helper to count parameters in the model function
    sig = inspect.signature(model_func)
    # subtract 1 because 'x' is the independent variable
    num_params = len(sig.parameters) - 1 
    
    def plotter(M_inf=0, A1=1.0, Gamma1=0.2, Omega1=9.24, gamma=0.01):
        
        # --- A. Setup Data & Guess ---
        plt.figure(figsize=(10, 6))
        
        # Plot Data
        plt.errorbar(t_data, M_data, yerr=M_err, fmt='o', color='gray', 
                     alpha=0.4, label='Data (Errors ignored)')
        
        # Construct guess list dynamically based on model args
        # (We provide enough defaults for y1, but truncate if using y2)
        full_guess_list = [M_inf, A1, Gamma1, Omega1, gamma]
        guess_params = full_guess_list[:num_params] 
        
        x_smooth = np.linspace(min(t_data), max(t_data), 300)
        
        # Plot Initial Guess (Blue)
        try:
            plt.plot(x_smooth, model_func(x_smooth, *guess_params), 'b--', 
                     alpha=0.7, label='Initial Guess')
        except:
            pass # Suppress plot error if slider params don't match model yet

        # --- B. Perform UNWEIGHTED Fit WITH BOUNDS ---
        try:
            # --- THE FIX IS HERE ---
            # We create bounds dynamically based on the number of parameters.
            # 1. Lower Bound: M_inf >= -1.5, All others >= 0
            lower_bounds = [-1.5] + [0] * (num_params - 1)
            
            # 2. Upper Bound: M_inf <= 1.5, All others <= 50 (prevents explosion to infinity)
            # 50 is chosen as a "safe" max for Rates/Freqs in your unit system.
            upper_bounds = [1.5] + [50] * (num_params - 1)
            
            # 3. Run Fit
            popt, pcov = curve_fit(
                model_func, 
                t_data, 
                M_data, 
                p0=guess_params, 
                maxfev=10000,
                bounds=(lower_bounds, upper_bounds) # <--- Apply Constraints
            )
            
            # --- C. Calculate Statistics ---
            residuals = M_data - model_func(t_data, *popt)
            ssr = np.sum(residuals**2)
            
            n = len(M_data)
            k = len(popt)
            bic = n * np.log(ssr / n) + k * np.log(n)
            
            # --- D. Plot Result ---
            plt.plot(x_smooth, model_func(x_smooth, *popt), 'r-', linewidth=2.5, label='Best Fit')
            
            plt.title(f"SSR: {ssr:.5f}  |  BIC: {bic:.2f} (Lower is better)", fontsize=14)
            print(f"--- FITTED PARAMETERS ({model_func.__name__}) ---")
            param_names = list(sig.parameters.keys())[1:]
            for name, val in zip(param_names, popt):
                print(f"{name:>10}: {val:.5f}")
                
        except Exception as e:
            plt.title(f"Fit failed: {e}", color='red')

        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

    # --- E. Sliders ---
    interact(plotter, 
             M_inf=FloatSlider(min=-1.0, max=1.0, step=0.01, value=0),
             A1=FloatSlider(min=0, max=2.0, step=0.01, value=1.0),
             Gamma1=FloatSlider(min=0, max=5, step=0.001, value=0.5),
             Omega1=FloatSlider(min=0, max=15, step=0.01, value=9.24),
             gamma=FloatSlider(min=0, max=2, step=0.001, value=0.01)
            )

# Usage
manual_fit_tool_unweighted(y1)

interactive(children=(FloatSlider(value=0.0, description='M_inf', max=1.0, min=-1.0, step=0.01), FloatSlider(v…

In addition, regarding the chi^2 analysis we discussed, I noticed that some of the data points have abnormally small error bars (as an example of what I mean, see the screenshot for a particular time series at 1/kFa = -0.01 and notice how small (0.00069) the M_err value is for the last data point (t=2098 us) compared to some of the other times...since chi^2 is a sum of squares of z-scores, and each z-score is inversely proportional to the error bar, these kinds of points get a lot of weight, optimizer tries really hard to fit them because mathematically that's what will minimize chi-squared.


In [ ]:
# --- Tool Setup Specifically for y2 ---
def fit_y2_tool():
    
    # y2 signature: (x, M_inf, Gamma_1, Omega_1)
    
    def plotter(M_inf=0, Gamma_1=0.2, Omega_1=9.24):
        
        # 1. Define the specific guess order for y2
        guess_params = [M_inf, Gamma_1, Omega_1]
        
        # 2. Setup Plot
        plt.figure(figsize=(10, 6))
        plt.errorbar(t_data, M_data, yerr=M_err, fmt='o', color='gray', alpha=0.4)
        
        x_smooth = np.linspace(min(t_data), max(t_data), 300)
        plt.plot(x_smooth, y2(x_smooth, *guess_params), 'b--', alpha=0.7, label='Initial Guess')

        # 3. Fit with Bounds (Dynamic based on len(guess_params))
        try:
            # Bounds: M_inf (-1.5 to 1.5), Others (0 to 50)
            lower_bounds = [-1.5] + [0]*(len(guess_params)-1)
            upper_bounds = [1.5] + [50]*(len(guess_params)-1)
            
            popt, pcov = curve_fit(y2, t_data, M_data, p0=guess_params, 
                                   maxfev=10000, bounds=(lower_bounds, upper_bounds))
            
            # 4. Stats
            residuals = M_data - y2(t_data, *popt)
            ssr = np.sum(residuals**2)
            n = len(M_data)
            k = len(popt)
            bic = n * np.log(ssr / n) + k * np.log(n)
            
            plt.plot(x_smooth, y2(x_smooth, *popt), 'r-', linewidth=2.5, label='Best Fit')
            plt.title(f"Model: y2  |  SSR: {ssr:.5f}  |  BIC: {bic:.2f}", fontsize=14)
            
            print(f"--- FITTED PARAMETERS (y2) ---")
            names = ["M_inf", "Gamma_1", "Omega_1"]
            for name, val in zip(names, popt):
                print(f"{name:>10}: {val:.5f}")
                
        except Exception as e:
            plt.title(f"Fit failed: {e}", color='red')

        plt.legend()
        plt.show()

    # 5. Sliders specifically for y2 variables
    interact(plotter, 
             M_inf=FloatSlider(min=-1.0, max=1.0, step=0.01, value=-0.06),
             Gamma_1=FloatSlider(min=0, max=2, step=0.001, value=0.5),
             Omega_1=FloatSlider(min=0, max=15, step=0.01, value=9.24)
            )

# Run it
fit_y2_tool()

interactive(children=(FloatSlider(value=-0.06, description='M_inf', max=1.0, min=-1.0, step=0.01), FloatSlider…

In [12]:
# --- Tool Setup Specifically for y3 ---
def fit_y3_tool():
    
    # y3 signature: (x, M_inf, A_1, Gamma_1, Omega_1, A_2, Gamma_2, Omega_2, gamma)
    
    def plotter(M_inf=-0.06, 
                A1=1.0, Gamma1=0.5, Omega1=9.24, 
                A2=0.5, Gamma2=0.5, Omega2=5.0, 
                gamma=0.01):
        
        # 1. Define the specific guess order for y3
        guess_params = [M_inf, A1, Gamma1, Omega1, A2, Gamma2, Omega2, gamma]
        
        # 2. Setup Plot
        plt.figure(figsize=(12, 7)) # Slightly larger for clarity
        plt.errorbar(t_data, M_data, yerr=M_err, fmt='o', color='gray', alpha=0.4, label='Data')
        
        x_smooth = np.linspace(min(t_data), max(t_data), 300)
        
        # Plot Initial Guess (Blue)
        try:
            plt.plot(x_smooth, y3(x_smooth, *guess_params), 'b--', alpha=0.6, label='Initial Guess')
        except:
            pass

        # 3. Fit with Bounds
        try:
            # Bounds: M_inf (-1.5 to 1.5), All others (0 to 50)
            # This prevents negative frequencies or amplitudes
            lower_bounds = [-1.5] + [0]*(len(guess_params)-1)
            upper_bounds = [ 1.5] + [50]*(len(guess_params)-1)
            
            popt, pcov = curve_fit(y3, t_data, M_data, p0=guess_params, 
                                   maxfev=15000, # Increased maxfev for complex model
                                   bounds=(lower_bounds, upper_bounds))
            
            # 4. Stats
            residuals = M_data - y3(t_data, *popt)
            ssr = np.sum(residuals**2)
            n = len(M_data)
            k = len(popt)
            bic = n * np.log(ssr / n) + k * np.log(n)
            
            plt.plot(x_smooth, y3(x_smooth, *popt), 'r-', linewidth=2.5, label='Best Fit')
            plt.title(f"Model: y3 (2-Freq)  |  SSR: {ssr:.5f}  |  BIC: {bic:.2f}", fontsize=14)
            
            print(f"--- FITTED PARAMETERS (y3) ---")
            names = ["M_inf", "A_1", "Gamma_1", "Omega_1", "A_2", "Gamma_2", "Omega_2", "gamma"]
            for name, val in zip(names, popt):
                print(f"{name:>10}: {val:.5f}")
                
        except Exception as e:
            plt.title(f"Fit failed: {e}", color='red')

        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

    # 5. Sliders specifically for y3
    # Note: Omega1 default is set to your known 9.24, Omega2 is set lower as a guess
    interact(plotter, 
             M_inf=FloatSlider(min=-1.0, max=1.0, step=0.01, value=-0.06),
             
             # Component 1
             A1=FloatSlider(min=0, max=2.0, step=0.01, value=1.0),
             Gamma1=FloatSlider(min=0, max=5, step=0.01, value=0.5),
             Omega1=FloatSlider(min=0, max=15, step=0.01, value=9.24),
             
             # Component 2
             A2=FloatSlider(min=0, max=2.0, step=0.01, value=0.2),
             Gamma2=FloatSlider(min=0, max=5, step=0.01, value=0.5),
             Omega2=FloatSlider(min=0, max=15, step=0.01, value=4.5),
             
             # Global Decay
             gamma=FloatSlider(min=0, max=2, step=0.001, value=0.01)
            )

# Run it
fit_y3_tool()

interactive(children=(FloatSlider(value=-0.06, description='M_inf', max=1.0, min=-1.0, step=0.01), FloatSlider…

In [ ]:
# --- Tool Setup Specifically for y4 ---
def fit_y4_tool():
    
    # y4 signature: (x, M_inf, A_1, Gamma_1, Omega_1, Gamma_2, Omega_2)
    # Note: A_2 is calculated automatically as (1 - A_1 + M_inf) inside the model
    
    def plotter(M_inf=-0.06, 
                A1=0.8, Gamma1=0.5, Omega1=9.24, 
                Gamma2=0.5, Omega2=4.5):
        
        # 1. Define the specific guess order for y4
        guess_params = [M_inf, A1, Gamma1, Omega1, Gamma2, Omega2]
        
        # 2. Setup Plot
        plt.figure(figsize=(12, 7))
        plt.errorbar(t_data, M_data, yerr=M_err, fmt='o', color='gray', alpha=0.4, label='Data')
        
        x_smooth = np.linspace(min(t_data), max(t_data), 300)
        
        # Plot Initial Guess (Blue)
        try:
            plt.plot(x_smooth, y4(x_smooth, *guess_params), 'b--', alpha=0.6, label='Initial Guess')
        except:
            pass

        # 3. Fit with Bounds
        try:
            # Bounds: M_inf (-1.5 to 1.5), All others (0 to 50)
            lower_bounds = [-1.5] + [0]*(len(guess_params)-1)
            upper_bounds = [ 1.5] + [50]*(len(guess_params)-1)
            
            popt, pcov = curve_fit(y4, t_data, M_data, p0=guess_params, 
                                   maxfev=15000, 
                                   bounds=(lower_bounds, upper_bounds))
            
            # 4. Stats
            residuals = M_data - y4(t_data, *popt)
            ssr = np.sum(residuals**2)
            n = len(M_data)
            k = len(popt)
            bic = n * np.log(ssr / n) + k * np.log(n)
            
            plt.plot(x_smooth, y4(x_smooth, *popt), 'r-', linewidth=2.5, label='Best Fit')
            plt.title(f"Model: y4 (Constrained 2-Freq)  |  SSR: {ssr:.5f}  |  BIC: {bic:.2f}", fontsize=14)
            
            print(f"--- FITTED PARAMETERS (y4) ---")
            names = ["M_inf", "A_1", "Gamma_1", "Omega_1", "Gamma_2", "Omega_2"]
            for name, val in zip(names, popt):
                print(f"{name:>10}: {val:.5f}")
            
            # Helper to see what the "Hidden" Amplitude A2 turned out to be
            implicit_A2 = 1 - popt[1] + popt[0] # 1 - A1 + Minf
            print(f"{'Implied A2':>10}: {implicit_A2:.5f}")

        except Exception as e:
            plt.title(f"Fit failed: {e}", color='red')

        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

    # 5. Sliders specifically for y4
    interact(plotter, 
             M_inf=FloatSlider(min=-1.0, max=1.0, step=0.01, value=-0.06),
             
             # Component 1 (Primary)
             A1=FloatSlider(min=0, max=2.0, step=0.01, value=0.8),
             Gamma1=FloatSlider(min=0, max=5, step=0.01, value=0.5),
             Omega1=FloatSlider(min=0, max=15, step=0.01, value=9.24),
             
             # Component 2 (Secondary)
             # Note: No A2 slider because it's mathematically locked to A1
             Gamma2=FloatSlider(min=0, max=5, step=0.01, value=0.5),
             Omega2=FloatSlider(min=0, max=15, step=0.01, value=4.5)
            )

# Run it
fit_y4_tool()

interactive(children=(FloatSlider(value=-0.06, description='M_inf', max=1.0, min=-1.0, step=0.01), FloatSlider…